In [76]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
# di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import hail as hl
hl.init() 

Running on Apache Spark version 2.4.1
SparkUI available at http://8c03623a3bed:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.28-61941242c15d
LOGGING: writing to /hail/large_vcf_filtering_annotation/hail-20191217-1338-0.2.28-61941242c15d.log


In [3]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()

import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

Loading BokehJS ...

In [63]:
mt = hl.read_matrix_table('large_vcf.mt')
mt = mt.select_rows(mt.rsid, mt.info.AC, mt.info.AF, mt.info.AN)
mt = hl.split_multi_hts(mt)

mt = mt.filter_cols(mt.s != 'WGS_139', keep = True)
mt = mt.filter_cols(mt.s != 'WGS_D6816', keep = True) #these samples should be out based on PCA performed in main notebook

## filter invervals that are included in repeatmasker

In [56]:
rpmk = hl.import_bed('repeatmasker', reference_genome='GRCh38', skip_invalid_intervals=True) # bed downloaded from https://genome.ucsc.edu/cgi-bin/hgTables on 16.12.19

2019-12-17 17:16:14 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (user-specified)
  Loading column 'f1' as type 'int32' (user-specified)
  Loading column 'f2' as type 'int32' (user-specified)
  Loading column 'f3' as type 'str' (user-specified)
  Loading column 'f4' as type 'str' (user-specified)
  Loading column 'f5' as type 'str' (type not specified)
  Loading column 'f6' as type 'str' (type not specified)
  Loading column 'f7' as type 'str' (type not specified)
  Loading column 'f8' as type 'str' (type not specified)
  Loading column 'f9' as type 'str' (type not specified)
  Loading column 'f10' as type 'str' (type not specified)
  Loading column 'f11' as type 'str' (type not specified)



In [79]:
rpmk.sample(0.0001).show()

2019-12-18 08:29:35 Hail: INFO: Ordering unsorted dataset with network shuffle


interval,target
interval<locus<GRCh38>>,str
[chr1:27872543-chr1:27886686),"""ENST00000328928.11"""
[chr1:247990267-chr1:247991205),"""ENST00000422727.2"""
[chr3:9917316-chr3:9933617),"""ENST00000413608.2"""
[chr3:106842907-chr3:107240594),"""ENST00000671678.1"""
[chr7:152134922-chr7:152177969),"""ENST00000360104.7"""
[chr9:33750679-chr9:33797925),"""ENST00000468152.2"""
[chr10:43387643-chr10:43396818),"""ENST00000498176.1"""
[chr10:100912963-chr10:100965135),"""ENST00000238961.9"""
[chr13:29918647-chr13:29926652),"""ENST00000450264.1"""


In [59]:
mt.count()

(21374902, 149)

In [64]:
mt = mt.filter_rows(hl.is_defined(rpmk[mt.locus]), keep = False)

In [65]:
mt.count()

2019-12-17 17:17:55 Hail: INFO: Ordering unsorted dataset with network shuffle


(9616373, 149)

In [66]:
hl.summarize_variants(mt)

2019-12-17 17:18:26 Hail: INFO: Ordering unsorted dataset with network shuffle


Number of alleles,Count
2,9616373
Allele type,Count
SNP,7471748
Deletion,1079407
Insertion,1065206
Complex,12
Contig,Count
chr1,816325
chr2,691023
chr3,537619


In [67]:
mt.sample_rows(0.000001).rsid.show()

2019-12-17 17:19:01 Hail: INFO: Ordering unsorted dataset with network shuffle


locus,alleles,rsid
locus<GRCh38>,array<str>,str
chr1:2982924,"[""T"",""C""]","""rs12046140"""
chr2:237951177,"[""C"",""CAAAAAACA""]",NA
chr3:13524830,"[""T"",""G""]","""rs2731346"""
chr4:95938127,"[""G"",""T""]","""rs113257899"""
chr5:17348618,"[""G"",""T""]",NA
chr5:84641607,"[""C"",""A""]",NA
chr9:121810490,"[""GA"",""G""]","""rs71835839"""
chr21:32052591,"[""A"",""G""]","""rs8130494"""


In [68]:
mt2 = hl.read_matrix_table('large_vcf.mt')
mt2 = mt2.select_rows(mt2.rsid, mt2.info.AC, mt2.info.AF, mt2.info.AN)
mt2 = hl.split_multi_hts(mt2)

mt2 = mt2.filter_cols(mt2.s != 'WGS_139', keep = True)
mt2 = mt2.filter_cols(mt2.s != 'WGS_D6816', keep = True) #these samples should be out based on PCA performed in main notebook

In [69]:
mt2 = mt2.filter_rows(hl.is_defined(rpmk[mt2.locus]), keep = True)

In [74]:
mt3 = mt2.filter_rows(hl.is_defined(rpmk[mt2.locus]), keep = False)

In [75]:
mt3.count()

2019-12-17 17:32:40 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-17 17:32:41 Hail: INFO: Ordering unsorted dataset with network shuffle


(0, 149)

In [70]:
mt2.count()

2019-12-17 17:20:02 Hail: INFO: Ordering unsorted dataset with network shuffle


(11758529, 149)

In [72]:
hl.summarize_variants(mt2)

2019-12-17 17:20:35 Hail: INFO: Ordering unsorted dataset with network shuffle


Number of alleles,Count
2,11758529
Allele type,Count
SNP,8650195
Insertion,1562006
Deletion,1546322
Complex,6
Contig,Count
chr1,943005
chr2,965588
chr3,836750


In [73]:
mt2.sample_rows(0.000001).rsid.show()

2019-12-17 17:21:12 Hail: INFO: Ordering unsorted dataset with network shuffle


locus,alleles,rsid
locus<GRCh38>,array<str>,str
chr1:93591168,"[""TAAAAAAA"",""T""]",NA
chr1:146758208,"[""T"",""A""]",NA
chr1:236244995,"[""T"",""C""]","""rs61833552"""
chr2:38964849,"[""T"",""A""]","""rs139228800"""
chr3:21532681,"[""CA"",""C""]",NA
chr3:127968493,"[""A"",""G""]",NA
chr4:177080602,"[""T"",""C""]","""rs2583201"""
chr5:71551094,"[""GTTGTTTAT"",""G""]",NA
chr11:60846873,"[""T"",""C""]","""rs537024"""


## import gnomADv3 coverage and annotate:

In [17]:
## code below is work in progress

In [90]:
gcov = hl.read_table('/hail/gnomad/gnomad.genomes.r3.0.coverage.ht/')

In [91]:
gcov.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'mean': float64 
    'median': int32 
    'count_array': array<int32> 
    'over_1': float32 
    'over_5': float32 
    'over_10': float32 
    'over_15': float32 
    'over_20': float32 
    'over_25': float32 
    'over_30': float32 
    'over_50': float32 
    'over_100': float32 
----------------------------------------
Key: ['locus']
----------------------------------------


In [ ]:
stats = gcov.aggregate(hl.agg.stats(gcov.over_5))

In [ ]:
stats

In [ ]:
gcov = gcov.select(gcov.over_5) #according to https://gnomad.broadinstitute.org/faq this is at least 5 reads

In [ ]:
mt = mt.annotate_rows(v3_coverage = gcov.over_5[mt.locus])

## import gnomADv3 table:

In [18]:
gnmd = hl.read_table('/hail/gnomad/gnomad.genomes.r3.0.sites.ht')

In [45]:
gnmd = gnmd.select(v3_nfe = gnmd.freq[2], 
            intergenic_consequences = gnmd.vep.intergenic_consequences,
            most_severe_consequence = gnmd.vep.most_severe_consequence,
            motif_feature_consequences = gnmd.vep.motif_feature_consequences,
            regulatory_feature_consequences = gnmd.vep.regulatory_feature_consequences,
            transcript_consequences = gnmd.vep.transcript_consequences,
            variant_class = gnmd.vep.variant_class,
            rsid = gnmd.rsid)

In [46]:
gnmd.v3_nfe.show()

locus,alleles,v3_nfe.AC,v3_nfe.AF,v3_nfe.AN,v3_nfe.homozygote_count
locus<GRCh38>,array<str>,int32,float64,int32,int32
chr1:10031,"[""T"",""C""]",0,0.00e+00,24292,0
chr1:10037,"[""T"",""C""]",0,0.00e+00,32798,0
chr1:10043,"[""T"",""C""]",0,0.00e+00,37164,0
chr1:10055,"[""T"",""C""]",0,0.00e+00,42048,0
chr1:10057,"[""A"",""C""]",2,3.98e-05,50204,0
chr1:10061,"[""T"",""C""]",0,0.00e+00,48358,0
chr1:10061,"[""T"",""TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCC""]",0,0.00e+00,48358,0
chr1:10064,"[""C"",""CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCAACCCTAACCCTAACCCTAACCCTAACCCTAACCCAAA""]",0,0.00e+00,63772,0
chr1:10067,"[""T"",""TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCC""]",0,0.00e+00,52294,0


In [5]:
gwas = hl.read_table('TS_Oct2018.mt')

In [49]:
stats = mt.aggregate_rows(hl.agg.stats(mt.gwas_p.P))

2019-12-16 13:36:50 Hail: INFO: Ordering unsorted dataset with network shuffle


In [50]:
stats

Struct(mean=0.4863693132405223, stdev=0.29268370026289625, min=2.112e-08, max=1.0, n=3634456, sum=1767687.8687228956)

In [6]:
gwas = gwas.annotate(variants = hl.parse_variant(hl.str(gwas.CHR)+':'+hl.str(gwas.BP)+':'+hl.str(gwas.A1)+':'+hl.str(gwas.A2), reference_genome='GRCh37'))

In [7]:
rg37 = hl.get_reference('GRCh37')  
rg38 = hl.get_reference('GRCh38')  
rg37.add_liftover('remple_qc.pyferences_grch37_to_grch38.over.chain.gz', rg38)  

In [8]:
gwas = gwas.annotate(new_locus=hl.liftover(gwas.variants.locus, 'GRCh38'))  
gwas = gwas.filter(hl.is_defined(gwas.new_locus))  
gwas = gwas.key_by(locus=gwas.new_locus, alleles=gwas.variants.alleles)  

In [9]:
gwas=gwas.select(gwas.SNP, gwas.P)

In [12]:
gwas.show()

2019-12-16 07:51:04 Hail: INFO: Ordering unsorted dataset with network shuffle


locus,alleles,SNP,P
locus<GRCh38>,array<str>,str,float64
chr1:796338,"[""T"",""C""]","""rs142557973""",7.29e-01
chr1:798969,"[""T"",""C""]","""rs141242758""",6.58e-01
chr1:800909,"[""A"",""T""]","""rs79010578""",8.88e-02
chr1:816376,"[""T"",""C""]","""rs143225517""",4.18e-01
chr1:817186,"[""A"",""G""]","""rs3094315""",2.16e-01
chr1:817341,"[""A"",""G""]","""rs3131972""",4.56e-01
chr1:817514,"[""T"",""C""]","""rs3131971""",1.43e-01
chr1:818025,"[""A"",""C""]","""rs61770173""",4.15e-01
chr1:818094,"[""C"",""G""]","""rs2073814""",5.46e-01


In [49]:
stats = mt.aggregate_rows(hl.agg.stats(mt.gwas_p.P))

2019-12-16 13:36:50 Hail: INFO: Ordering unsorted dataset with network shuffle


In [50]:
stats

Struct(mean=0.4863693132405223, stdev=0.29268370026289625, min=2.112e-08, max=1.0, n=3634456, sum=1767687.8687228956)

In [44]:
mt = mt.annotate_rows(gwas_p = gwas[mt.row_key])

In [35]:
mtf = mt.filter_rows((mt.gwas_p.P < 5e10-8) & (mt.gwas_p.P != 0))

In [36]:
mtf.count_rows()

2019-12-16 08:55:01 Hail: INFO: Ordering unsorted dataset with network shuffle


3634456